## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boundiali,CI,9.5217,-6.4869,66.45,16,3,3.56,clear sky
1,1,Kruisfontein,ZA,-34.0033,24.7314,61.29,77,0,4.03,clear sky
2,2,Luderitz,NaN,-26.6481,15.1594,74.10,69,35,5.95,scattered clouds
3,3,Peniche,PT,39.3558,-9.3811,56.08,93,81,6.42,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,66.88,82,67,3.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature (degree Faranhite) you would like for your trip? "))
max_temp=float(input("What is the maximum temperature (degree Faranhite) you would like for your trip? "))

What is the minimum temperature (degree Faranhite) you would like for your trip? 50
What is the maximum temperature (degree Faranhite) you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df=city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_city_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boundiali,CI,9.5217,-6.4869,66.45,16,3,3.56,clear sky
1,1,Kruisfontein,ZA,-34.0033,24.7314,61.29,77,0,4.03,clear sky
2,2,Luderitz,NaN,-26.6481,15.1594,74.10,69,35,5.95,scattered clouds
3,3,Peniche,PT,39.3558,-9.3811,56.08,93,81,6.42,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,66.88,82,67,3.00,broken clouds
...,...,...,...,...,...,...,...,...,...,...
653,653,Mgandu,TZ,-5.9500,34.1333,61.75,90,100,1.36,overcast clouds
657,657,Bouna,CI,9.2667,-3.0000,70.70,17,17,5.03,few clouds
658,658,Elliot,ZA,-31.3333,27.8500,59.68,53,6,5.08,clear sky
659,659,Kambove,CD,-10.8764,26.5969,58.95,97,100,2.84,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294 entries, 0 to 660
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              294 non-null    int64  
 1   City                 294 non-null    object 
 2   Country              286 non-null    object 
 3   Lat                  294 non-null    float64
 4   Lng                  294 non-null    float64
 5   Max Temp             294 non-null    float64
 6   Humidity             294 non-null    int64  
 7   Cloudiness           294 non-null    int64  
 8   Wind Speed           294 non-null    float64
 9   Current Description  294 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 25.3+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_city_df.dropna()


In [7]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286 entries, 0 to 660
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              286 non-null    int64  
 1   City                 286 non-null    object 
 2   Country              286 non-null    object 
 3   Lat                  286 non-null    float64
 4   Lng                  286 non-null    float64
 5   Max Temp             286 non-null    float64
 6   Humidity             286 non-null    int64  
 7   Cloudiness           286 non-null    int64  
 8   Wind Speed           286 non-null    float64
 9   Current Description  286 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 24.6+ KB


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boundiali,CI,66.45,clear sky,9.5217,-6.4869,
1,Kruisfontein,ZA,61.29,clear sky,-34.0033,24.7314,
3,Peniche,PT,56.08,broken clouds,39.3558,-9.3811,
4,Cape Town,ZA,66.88,broken clouds,-33.9258,18.4232,
5,Sechura,PE,72.43,broken clouds,-5.5569,-80.8222,
7,Puerto Baquerizo Moreno,EC,74.88,overcast clouds,-0.9000,-89.6000,
9,Ayacucho,PE,56.46,overcast clouds,-13.1583,-74.2239,
11,Ushuaia,AR,55.06,light intensity drizzle,-54.8000,-68.3000,
14,Saint-Louis,SN,68.95,broken clouds,16.3333,-15.0000,
16,East London,ZA,71.29,clear sky,-33.0153,27.9116,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boundiali,CI,66.45,clear sky,9.5217,-6.4869,Hotel Paysan Boundiali
1,Kruisfontein,ZA,61.29,clear sky,-34.0033,24.7314,Oyster Bay House Rental
3,Peniche,PT,56.08,broken clouds,39.3558,-9.3811,Hotel Star inn Peniche
4,Cape Town,ZA,66.88,broken clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
5,Sechura,PE,72.43,broken clouds,-5.5569,-80.8222,Hospedaje Costa Bella


In [11]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl> 
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))